http://deap.readthedocs.io/en/master/examples/gp_symbreg.html
https://github.com/DEAP/deap/blob/4db155fb3c4fe1678f8d7cd03a638248a1a2f447/examples/gp/symbreg.py

In [2]:
import operator
import math
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [3]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [7]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [11]:
def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [12]:
def main():
    random.seed(318)
    
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("arg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats, 
                                  halloffame=hof, verbose=True)
    return(pop, log, hof)

In [15]:
if __name__ == "__main__":
    pop, log, hof = main()

   	      	                fitness                	              size             
   	      	---------------------------------------	-------------------------------
gen	nevals	arg    	max  	min     	std    	arg    	max	min	std    
0  	300   	1.78879	30.34	0.450825	2.67896	3.54667	7  	2  	1.49482
1  	166   	1.43254	44.4437	0.183711	3.05668	3.60667	12 	1  	1.77725
2  	166   	2.16879	315.736	0.165572	18.1873	3.55   	9  	1  	1.62506
3  	163   	0.98255	2.9829 	0.165572	0.712666	3.42667	9  	1  	1.45073
4  	153   	0.836017	14.538 	0.165572	0.979399	3.77   	11 	1  	1.64025
5  	158   	0.944635	18.9739	0.165572	1.61614 	3.77667	10 	1  	1.62894
6  	169   	0.885819	14.2181	0.165572	1.00296 	4      	10 	1  	1.87617
7  	167   	0.731332	3.35292	0.165572	0.56016 	4.35   	10 	1  	1.92722
8  	187   	0.785916	19.1852	0.13008 	1.2426  	5.13667	14 	1  	2.21465
9  	176   	0.672788	14.2339	0.123719	1.00764 	5.71667	14 	1  	2.48389
10 	176   	0.786975	32.4952	0.123719	2.56679 	6.27333	15 	1  	2.33922
11 	167

In [17]:
hof

In [24]:
hof.items

[[<deap.gp.Primitive at 0x7f662629fe58>,

In [25]:
pop

[[<deap.gp.Primitive at 0x7f662629fe58>,

In [26]:
log

[{'gen': 0, 'nevals': 300},
 {'gen': 1, 'nevals': 166},
 {'gen': 2, 'nevals': 166},
 {'gen': 3, 'nevals': 163},
 {'gen': 4, 'nevals': 153},
 {'gen': 5, 'nevals': 158},
 {'gen': 6, 'nevals': 169},
 {'gen': 7, 'nevals': 167},
 {'gen': 8, 'nevals': 187},
 {'gen': 9, 'nevals': 176},
 {'gen': 10, 'nevals': 176},
 {'gen': 11, 'nevals': 167},
 {'gen': 12, 'nevals': 170},
 {'gen': 13, 'nevals': 173},
 {'gen': 14, 'nevals': 163},
 {'gen': 15, 'nevals': 178},
 {'gen': 16, 'nevals': 154},
 {'gen': 17, 'nevals': 177},
 {'gen': 18, 'nevals': 177},
 {'gen': 19, 'nevals': 175},
 {'gen': 20, 'nevals': 167},
 {'gen': 21, 'nevals': 159},
 {'gen': 22, 'nevals': 161},
 {'gen': 23, 'nevals': 165},
 {'gen': 24, 'nevals': 188},
 {'gen': 25, 'nevals': 152},
 {'gen': 26, 'nevals': 164},
 {'gen': 27, 'nevals': 159},
 {'gen': 28, 'nevals': 164},
 {'gen': 29, 'nevals': 174},
 {'gen': 30, 'nevals': 141},
 {'gen': 31, 'nevals': 154},
 {'gen': 32, 'nevals': 165},
 {'gen': 33, 'nevals': 173},
 {'gen': 34, 'nevals': 1